In [387]:
import numpy as np 
import pandas as pd 
import portion as P
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable 
from math import prod


# Transforming LCL:s to linear problems

Goal is to find $\alpha, \beta, a,b,c...$ such that if some $A, A, B$ is possible for active/passive nodes, then the corresponding $a+a+b \leq \beta / \geq \alpha$. Additionally it would be great to see if these values $a, b, c...$ could be expanded to intervals.

Example: problem

```
A AB AB

B AB AB
```
can be interpreted as $\alpha = 1, \beta=2, A = \left[ 0,\frac{1}{3}\right), B =  \left(\frac{2}{3}, 1\right]$.

What is the system of linear inequalities that could imply this?

\begin{align*}
3a &\leq \beta \\
2a+b &\leq \beta \\
a+2b &\leq \beta \\
3b &> \beta \\
\\
3a &< \alpha \\
2a+b &\geq \alpha \\
a+2b &\geq \alpha \\
3b &\geq \alpha

\end{align*}

In [388]:
# Initialize variables

# Add active and passive constraints in RE-formalism
active = """
D ABCD ABCD
C BC ABC
"""
passive = """
A ABCD ABCD
B BC BCD
"""

# Enable/disable debug mode
debug = True

In [389]:
actives = list(map(lambda x: x.split(), active.strip().split("\n")))
passives = list(map(lambda x: x.split(), passive.strip().split("\n")))

variables = sorted(list(set(list("".join("".join([active, passive]).split())))))

combinations = pd.DataFrame({"combination": itertools.combinations_with_replacement(variables, len(passives[0]))})
combinations["active"] = False
combinations["passive"] = False

for row in passives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "passive"] = True 

for row in actives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "active"] = True 

if debug: print(combinations)


   combination  active  passive
0    (A, A, A)   False     True
1    (A, A, B)   False     True
2    (A, A, C)   False     True
3    (A, A, D)    True     True
4    (A, B, B)   False     True
5    (A, B, C)    True     True
6    (A, B, D)    True     True
7    (A, C, C)    True     True
8    (A, C, D)    True     True
9    (A, D, D)    True     True
10   (B, B, B)   False     True
11   (B, B, C)    True     True
12   (B, B, D)    True     True
13   (B, C, C)    True     True
14   (B, C, D)    True     True
15   (B, D, D)    True    False
16   (C, C, C)    True    False
17   (C, C, D)    True    False
18   (C, D, D)    True    False
19   (D, D, D)    True    False


In [390]:

epsilon = 0.001

model = LpProblem(name="Reductions", sense=LpMaximize)

alpha = LpVariable(name = "alpha", lowBound=0, upBound=10)

beta = LpVariable(name = "beta", lowBound=0, upBound=10)

pulp_variables = dict(zip(variables, [LpVariable(name = v, lowBound=0, upBound=1) for v in variables]))

# Try different targets in order to get some readable solutions
model += beta-alpha

# Add initial constraints
#for v in pulp_variables.values():
#    model += (v>=0, f"{v}, low")
#    model += (v<=1, f"{v}, high")

# Add constraints

for index, row in combinations.iterrows():
    if row["active"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=alpha, f"{index}_Active")
    else:
        model += (sum(pulp_variables[c] for c in row["combination"])<= alpha-epsilon, f"{index}_not_Active")

    if row["passive"]:
        model += (sum(pulp_variables[c] for c in row["combination"])<=beta, f"{index}_Passive")
    else:
        model += (sum(pulp_variables[c] for c in row["combination"])>=beta+epsilon, f"{index}_not_Passive")
    

    
if debug: print(model)

if model.solve() == 1:
    print("Linear model found:")

    for var in model.variables():
        print(var, var.value())

else: print("No suitable model found")

Reductions:
MAXIMIZE
-1*alpha + 1*beta + 0
SUBJECT TO
0_not_Active: 3 A - alpha <= -0.001

0_Passive: 3 A - beta <= 0

1_not_Active: 2 A + B - alpha <= -0.001

1_Passive: 2 A + B - beta <= 0

2_not_Active: 2 A + C - alpha <= -0.001

2_Passive: 2 A + C - beta <= 0

3_Active: 2 A + D - alpha >= 0

3_Passive: 2 A + D - beta <= 0

4_not_Active: A + 2 B - alpha <= -0.001

4_Passive: A + 2 B - beta <= 0

5_Active: A + B + C - alpha >= 0

5_Passive: A + B + C - beta <= 0

6_Active: A + B + D - alpha >= 0

6_Passive: A + B + D - beta <= 0

7_Active: A + 2 C - alpha >= 0

7_Passive: A + 2 C - beta <= 0

8_Active: A + C + D - alpha >= 0

8_Passive: A + C + D - beta <= 0

9_Active: A + 2 D - alpha >= 0

9_Passive: A + 2 D - beta <= 0

10_not_Active: 3 B - alpha <= -0.001

10_Passive: 3 B - beta <= 0

11_Active: 2 B + C - alpha >= 0

11_Passive: 2 B + C - beta <= 0

12_Active: 2 B + D - alpha >= 0

12_Passive: 2 B + D - beta <= 0

13_Active: B + 2 C - alpha >= 0

13_Passive: B + 2 C - beta <= 0

1